In [2]:
# Importing the required modules.
import random
import json
import pickle
import numpy as np
import nltk

from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.models import load_model

from sklearn.model_selection import train_test_split
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [1]:
lemm = WordNetLemmatizer() # Create a WordNetLemmatizer() class object.

meaning = json.loads(open("dataset.json").read()) # Read the contents from the “dataset.json” file and store it to a variable "meaning".

# Creating empty lists to store contents of dataset.json.
vocab = []
groups = []
docs = []
char_ignore = ["?", "!", ".", ","]

# Iterate through the entries in meaning['intents'] and process the patterns.
for entry in meaning['intents']:
	for ptrn in entry['patterns']:
		# separating words from patterns and adding them to word_list
		# word_tokenize(sentence) takes a sentence as a parameter 
		# and then returns a list containing all the words of the sentence as strings.
		word_list = nltk.word_tokenize(ptrn) 
		vocab.extend(word_list) 
		
		# associating patterns with respective tags, in the form of a tuple
		docs.append(((word_list), entry['tag']))

		# appending the tags to the groups list
		if entry['tag'] not in groups:
			groups.append(entry['tag'])

# Modify vocab to store the root words/lemma using a list comprehension.
vocab = [lemm.lemmatize(word) for word in vocab if word not in char_ignore]
vocab = sorted(set(vocab))

NameError: name 'WordNetLemmatizer' is not defined

In [5]:
# Dump the data from vocab and groups to binary files.
pickle.dump(vocab, open('vocab.pkl', 'wb'))
pickle.dump(groups, open('groups.pkl', 'wb'))

In [6]:
len(vocab)

174

In [7]:
# we need numerical values of the words because a neural network needs numerical values to work with

training = [] # empty list to store the data used for training
output_empty = [0]*len(groups) # store as many 0's as there are groups/tags in dataset.json

# Now we'll create a bag that will store the 0's and 1's.
# 0 - word isn't in the pattern, 1 - word is in the pattern.
# We'' do this by iterating through the documents list 
# and appending 1 to bag if the word is in patterns, 0 otherwise.
for document in docs:
	bag = []
	patterns = document[0]
	patterns = [lemm.lemmatize(word.lower()) for word in patterns]
	for word in vocab:
		bag.append(1) if word in patterns else bag.append(0)
		
	output_row = list(output_empty) # Make a copy of output_empty.
	output_row[groups.index(document[1])] = 1
	training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

# split the training set consisting of 1's and 0's into 2 parts: train_x and train_y
training_x = list(training[:, 0])
training_y = list(training[:, 1])
training_x = np.array(training_x)
training_y = np.array(training_y)
train_x, test_x, train_y, test_y = train_test_split(training_x, training_y, test_size=0.25, random_state=42)

C:\Users\Franc\AppData\Local\Temp\ipykernel_18496\3893058225.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
len(train_x)

78

In [9]:
len(train_y)

78

In [10]:
# Now we've reached the model-building stage.
# We'll deploy a Sequential machine learning model and train it on the dataset prepared above.
botmodel = Sequential()
botmodel.add(Dense(96, input_shape=(len(train_x[0]), ), activation='relu')) # add() is used to add layers in a neural network
botmodel.add(Dropout(0.3)) # dropout() is used to avoid overfitting
botmodel.add(Dense(52, activation='relu'))
botmodel.add(Dropout(0.2))
botmodel.add(Dense(26, activation='relu'))
botmodel.add(Dropout(0.2))
botmodel.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
# Compile the model.
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
botmodel.compile(loss='categorical_crossentropy',
			optimizer=sgd, metrics=['accuracy'])
hist = botmodel.fit(np.array(train_x), np.array(train_y),
				epochs=1000, batch_size=5, verbose=0)

c:\Users\Franc\.pyenv\pyenv-win\versions\3.11.2\Lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
botmodel = load_model('chatbotmodel.h5')

In [13]:
res = botmodel.predict(np.array([test_x[0]]))[0]
res

NameError: name 'test_x' is not defined

In [14]:
ERROR = 0.01
results = [[i, r] for i, r in enumerate(res) if r > ERROR] # append from res if the value is greater than ERROR
results.sort(key=lambda x: x[1], reverse=True) # sort results
print("predictions:")
print(results)
print("ground truth")
print(np.where(test_y[0] == 1))

NameError: name 'res' is not defined

In [15]:
right = 0
wrong = 0
almost = 0
ERROR = 0.01

for i, test in enumerate(test_x):
    res = botmodel.predict(np.array([test]))[0]
    results = [[i, r] for i, r in enumerate(res) if r > ERROR] # append from res if the value is greater than ERROR
    results.sort(key=lambda x: x[1], reverse=True) # sort results
    
    closeResults = [i[0] for i in results]
    gt = np.where(test_y[i] == 1)[0]
    if results[0][0] == gt:
        right += 1
    elif gt in closeResults:
        print(results)
        almost += 1
    else:
        wrong += 1

print("Right: {}".format(right))
print("Wrong: {}".format(wrong))
print("Almost: {}".format(almost))

NameError: name 'test_x' is not defined

In [42]:

# Save the model to an h5 file.
botmodel.save("chatbotmodel.h5", hist)